In [1]:
%matplotlib inline

# Process RAW Astro Images

## Configuration information

In [2]:
OffsetsDir = r'/home/jean-phi/Projects/Astrolights/Test_Images/Lagoon/BIAS'
FlatsDir = r'/home/jean-phi/Projects/Astrolights/Test_Images/Lagoon/FLAT'
DarksDir = r'/home/jean-phi/Projects/Astrolights/Test_Images/Lagoon/DARK'
LightsDir = r'/home/jean-phi/Projects/Astrolights/Test_Images/Lagoon/LIGHT'

parameters = {'OFFSET': OffsetsDir, 'FLAT':FlatsDir, 'DARK':DarksDir, 'LIGHT':LightsDir}

## Master Generation

### Master Offset

In [3]:
import os
import re
import rawpy
import imageio
import numpy as np

RAW_FORMAT = ['CR2', 'NEF']
VALID_FILE_EXTENTION = RAW_FORMAT + ['FIT', 'ASDF']

def buildValidFileExtentionRegex(validFileExtentionArray):
    resRegex = r'|'
    resRegex = r'^.*\.(' + resRegex.join(validFileExtentionArray) + ')$'
    
    return resRegex

def getFileList(method='directory', parameters={}):
    filesType = ['OFFSET', 'FLAT', 'DARK', 'LIGHT']
    files = {}
    # TODO: Add regex for accepted file extentions
    matchFiles = re.compile(buildValidFileExtentionRegex(VALID_FILE_EXTENTION))
    if method == 'directory':
        for fileType in filesType:
            if fileType in parameters:
                files[fileType] = [os.path.join(parameters[fileType], f) for f in os.listdir(parameters[fileType]) if matchFiles.match(f)]
    else:
        raise ValueError("Method {}: Import unkown import method".format(method))
    
    return files

In [4]:
listFiles = getFileList(parameters=parameters)

In [ ]:
%timeit
tobeTreated = listFiles['OFFSET'][0:15]
with rawpy.imread(tobeTreated[0]) as raw:
        #rgb = raw.postprocess()
        #rgb = raw.postprocess(gamma=(1,1), no_auto_bright=True, output_bps=16)
        firstImage = raw.postprocess(no_auto_bright=True, output_bps=16)

res = np.empty(firstImage.shape + (len(tobeTreated),), np.int32)
res[:,:,:,0] = firstImage
cpt = 1
        
for f in tobeTreated[1:]:
    with rawpy.imread(f) as raw:
        #rgb = raw.postprocess()
        #rgb = raw.postprocess(gamma=(1,1), no_auto_bright=True, output_bps=16)
        res[:,:,:,cpt] = raw.postprocess(no_auto_bright=True, output_bps=16)
        cpt += 1

In [ ]:
res.shape

In [ ]:
res.size

In [ ]:
resMedian = np.median(res, axis=3)

In [ ]:
resMedian.shape

In [ ]:
resMedian

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(resMedian)
plt.show()